In [6]:
#Colab setting
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/DSAIL')

import os
os.chdir('/content/drive/My Drive/DSAIL')

Mounted at /content/drive


In [9]:
import numpy as np
# 데이터셋을 continous하게 만들었다.
# 언제나 드는 의문은... user랑 item idx가 continous하지 않은데 보통 index 값을 훈련에 쓰지않나?

test = np.load("continous_test.npy")
train = np.load("continous_train.npy")

# GPU
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print("device", device)


In [10]:
# 필요한 함수들
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random

class BPR():
  def __init__(self, train_R, test_R, d=16, lamb = 0.01, learning_rate=1e-3, epochs= 100):
    '''
    BPR + KNN version
    '''
    self.num_user, self.num_item = train_R.shape
    self.test_n, self.test_m = test_R.shape

    self.latent_dimension = d

    # implicit하게 변환
    self.train_R = (train_R > 0).astype(float)
    self.test_R = (test_R > 0).astype(float)

    self.lr = learning_rate
    self.epoch = epochs
    self.lamb = lamb

    # item symmetric item-correlation learning이 목적 !
    self.C = np.random.standard_normal((self.num_item, self.num_item))
    # self.AUC_matrix = np.zeros((self.num_item, self.num_item))

    self.pos_items_dict = {user: np.where(self.train_R[user, :] > 0)[0] for user in range(self.num_user)}
    self.neg_items_dict = {user: np.where(self.train_R[user, :] == 0)[0] for user in range(self.num_user)}

    self.lj = None
    self.li = None

  def gradient_sigmoid(self, x):
    return sigmoid(x) * np.exp(-x)

  # dBPR-OPT/dtheta
  def gradient(self, u, i, j):
    # -sigma_term * dxthat/dtheta - selflamb * theta
    xuij = self.get_xuij(u, i, j)
    sigma_term = self.gradient_sigmoid(xuij)
    pos_items = self.pos_items_dict[u]
    neg_items = self.neg_items_dict[u]
    self.li = list(set(pos_items) - set([i]))
    self.lj = list(set(neg_items) - set([j]))

    dCi = -sigma_term * 1 - self.lamb * self.C[i, self.li]
    dCj = sigma_term - self.lamb * self.C[j, self.lj]
    return dCi, dCj

  def get_xuij(self, u, i, j):
    pos_items = self.pos_items_dict[u]
    neg_items = self.neg_items_dict[u]
    excluded_value = self.C[i][i]
    xui = np.sum(self.C[i][pos_items]) - excluded_value
    excluded_value = self.C[j][j]
    xuj = np.sum(self.C[j][neg_items]) - excluded_value
    # overflow 방지
    cc = np.clip(xui - xuj, -709.78, 709.78)
    return cc

  def gradient_descent(self, u, i, j):
    dCi, dCj = self.gradient(u, i, j)
    self.C[u, self.li] -= self.lr * dCi
    self.C[i, self.lj] -= self.lr * dCj


  def compute_AUC(self):
    auc = 0
    for u in range(self.num_user):
      pos_items = self.pos_items_dict[u]
      neg_items = self.neg_items_dict[u]
      for i in pos_items:
        for j in neg_items:
          if np.sum(self.C[i][pos_items]) - self.C[i][i] > np.sum(self.C[j][neg_items]) - self.C[j][j]:
            auc += 1
    auc /= (len(pos_items) * len(neg_items))

    return auc

  def fit(self):
    train_auc_list = []

    for epoch in range(self.epoch):
      # (u, i, j) random하게 sampling해야 item-wised user-wised update 피할 수 있음 (bootstrap)
      # user는 iterate방식으로 가고, i, j만 샘플링해도 될듯!
      user_AUC = 0
      for u in range(self.num_user):
        # user의 positive item pair = i 후보
        pos_items = np.where(self.train_R[u, :] > 0)[0]
        neg_items = np.where(self.train_R[u, :] == 0)[0]

        # cold_user의 경우 pass
        if len(pos_items) == 0:
          continue

        i = np.random.choice(pos_items)
        j = np.random.choice(neg_items)

        # user마다 iter 돌면서 i, j 랜덤하게 뽑고 update
        self.gradient_descent(u, i, j)

      if epoch == 10000 or epoch == 20000 or epoch == 29999:
        user_AUC = self.compute_AUC()

        user_AUC /= self.num_user
        train_auc_list.append(user_AUC)
        print(f'Epoch [{epoch}/{self.epoch}], train_AUC: {user_AUC}')

    return train_auc_list


In [ ]:
import matplotlib.pyplot as plt

model = BPR(train_R = train, test_R = test, d=15, epochs= 30000)

# Train the model
train_auc_list = model.fit()
plt.plot(train_auc_list)

# Error
RuntimeWarning: overflow encountered in exp
  return sigmoid(x) * np.exp(-x)

  float.64 -> np.clip(xui - xuj, -709.78, 709.78)